The Following code will locate the address of Newport Pier and identify open locations nearby that have trash receptacles. 

This data was purely found using the Foursquare API looking nearby locations the the beach in question. 

There is no clustering as the directions are to locate nearby open locations by the beach which have trash receiving areas. 

In [1]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


In [2]:
CLIENT_ID = '2ZUEYD1EBQ2DYJK4I1JGWVTGDHTYSB54P3ZZUC2FSST1ADVU' # your Foursquare ID
CLIENT_SECRET = '3KD5JKGKYUL30OCEWSC2GS2LRP33VA4TCMXINX4SERFBQBJR' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2ZUEYD1EBQ2DYJK4I1JGWVTGDHTYSB54P3ZZUC2FSST1ADVU
CLIENT_SECRET:3KD5JKGKYUL30OCEWSC2GS2LRP33VA4TCMXINX4SERFBQBJR


In [3]:
address = '70 Newport Pier, Newport Beach, CA'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


33.6076172120154 -117.928736994079


In [4]:
search_query = ''
radius = 500
print(search_query + ' .... OK!')

 .... OK!


In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=2ZUEYD1EBQ2DYJK4I1JGWVTGDHTYSB54P3ZZUC2FSST1ADVU&client_secret=3KD5JKGKYUL30OCEWSC2GS2LRP33VA4TCMXINX4SERFBQBJR&ll=33.6076172120154,-117.928736994079&v=20180604&query=&radius=500&limit=30'

In [11]:
results = requests.get(url).json()


In [7]:
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1e2941735', 'name': 'B...",False,4c719ee3d944224bfab20a29,NaN,US,Newport Beach,United States,NaN,81,"[Newport Beach, CA 92663, United States]","[{'label': 'display', 'lat': 33.60833409148612...",33.608334,-117.928891,NaN,92663,CA,Newport Beach Boardwalk,v-1537903888
1,"[{'id': '4bf58dd8d48988d1e2941735', 'name': 'B...",False,4e500f752271a1bdc3e7af97,21st Street,US,Newport Beach,United States,West Oceanfront,74,"[21st Street (West Oceanfront), Newport Beach,...","[{'label': 'display', 'lat': 33.60828636458094...",33.608286,-117.928673,NaN,92663,CA,21st Street Beach,v-1537903888
2,"[{'id': '4bf58dd8d48988d166941735', 'name': 'S...",False,53c00cf3498e4340417b9574,NaN,US,Newport Beach,United States,NaN,44,"[Newport Beach, CA, United States]","[{'label': 'display', 'lat': 33.608016, 'lng':...",33.608016,-117.928802,NaN,NaN,CA,McFadden Square,v-1537903888
3,"[{'id': '4bf58dd8d48988d10e951735', 'name': 'F...",False,5ae606a3625a66002c422ba9,NaN,US,Newport Beach,United States,NaN,39,"[Newport Beach, CA 92663, United States]","[{'label': 'display', 'lat': 33.607967, 'lng':...",33.607967,-117.928710,Newport Pier,92663,CA,West Caught Fish Co.,v-1537903888
4,"[{'id': '4bf58dd8d48988d1c9941735', 'name': 'I...",False,4c310936ed37a59339c96903,2010 W Oceanfront,US,Newport Beach,United States,balboa blvd,39,"[2010 W Oceanfront (balboa blvd), Newport Beac...","[{'label': 'display', 'lat': 33.60772650100333...",33.607727,-117.928336,NaN,92663,CA,Pipeline,v-1537903888


In [8]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Newport Beach Boardwalk,Beach,NaN,US,Newport Beach,United States,NaN,81,"[Newport Beach, CA 92663, United States]","[{'label': 'display', 'lat': 33.60833409148612...",33.608334,-117.928891,NaN,92663,CA,4c719ee3d944224bfab20a29
1,21st Street Beach,Beach,21st Street,US,Newport Beach,United States,West Oceanfront,74,"[21st Street (West Oceanfront), Newport Beach,...","[{'label': 'display', 'lat': 33.60828636458094...",33.608286,-117.928673,NaN,92663,CA,4e500f752271a1bdc3e7af97
2,McFadden Square,Sculpture Garden,NaN,US,Newport Beach,United States,NaN,44,"[Newport Beach, CA, United States]","[{'label': 'display', 'lat': 33.608016, 'lng':...",33.608016,-117.928802,NaN,NaN,CA,53c00cf3498e4340417b9574
3,West Caught Fish Co.,Fish Market,NaN,US,Newport Beach,United States,NaN,39,"[Newport Beach, CA 92663, United States]","[{'label': 'display', 'lat': 33.607967, 'lng':...",33.607967,-117.928710,Newport Pier,92663,CA,5ae606a3625a66002c422ba9
4,Pipeline,Ice Cream Shop,2010 W Oceanfront,US,Newport Beach,United States,balboa blvd,39,"[2010 W Oceanfront (balboa blvd), Newport Beac...","[{'label': 'display', 'lat': 33.60772650100333...",33.607727,-117.928336,NaN,92663,CA,4c310936ed37a59339c96903
5,Lifeguard Headquarters Newport Beach,Beach,NaN,US,NaN,United States,NaN,74,"[California, United States]","[{'label': 'display', 'lat': 33.60763889983848...",33.607639,-117.929543,NaN,NaN,California,4da1316363b5a35deafce319
6,Dory Fishing Fleet Market,Beach,110 McFadden Pl,US,Newport Beach,United States,Balboa Blvd.,47,"[110 McFadden Pl (Balboa Blvd.), Newport Beach...","[{'label': 'display', 'lat': 33.60798238996423...",33.607982,-117.929008,NaN,92663,CA,4da12e702939b1f7f072f056
7,Newport Beach Pier,Pier,1 Newport Pier,US,Newport Beach,United States,at 21st St,47,"[1 Newport Pier (at 21st St), Newport Beach, C...","[{'label': 'display', 'lat': 33.60769453625274...",33.607695,-117.929245,NaN,92663,CA,4a790799f964a520dce61fe3
8,Boardwalk Sports,Sporting Goods Shop,NaN,US,Newport Beach,United States,NaN,45,"[Newport Beach, CA 92663, United States]","[{'label': 'display', 'lat': 33.60795789734428...",33.607958,-117.928472,NaN,92663,CA,4c0adb30340720a14d3c8793
9,Diamond Parking Lot,Parking,NaN,US,Newport Beach,United States,NaN,25,"[Newport Beach, CA, United States]","[{'label': 'display', 'lat': 33.60748108367527...",33.607481,-117.928959,NaN,NaN,CA,4e0e342114956c6284f81d1d


In [9]:
dataframe_filtered.name

0                  Newport Beach Boardwalk
1                        21st Street Beach
2                          McFadden Square
3                     West Caught Fish Co.
4                                 Pipeline
5     Lifeguard Headquarters Newport Beach
6                Dory Fishing Fleet Market
7                       Newport Beach Pier
8                         Boardwalk Sports
9                      Diamond Parking Lot
10                  Newport Coffee Company
11                        Balboa Peninsula
12                         Charlie's Chili
13                Newport Pier Marketplace
14                       20th Street Beach
15                      Lifeguard Tower 20
16                       19th Street Beach
17                             Hoag Irvine
18                    2004 West Oceanfront
19                               My office
20                         Jane's Corndogs
21                          PJ's Surfrider
22                          The Beach Shop
23         

In [10]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Newport Beach',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map